In [1]:
1

1

In [2]:
cd ..

/home/ayush/Documents/AI/Projects/Fake-news-detector


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvalutionConfig:
    root_dir: Path
    model_path :  Path
    tokenizer_path: Path
    prediction_results : Path

In [4]:
from src.constants import * 
from src.utils.common import read_yaml_file,create_dir 

In [5]:
class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,
                 param=PARAM_FILE_PATH):
        self.config=read_yaml_file(config)
        self.param=read_yaml_file(param)
        create_dir([self.config.root_Artifact])

    def getModelEvalution(self)->ModelEvalutionConfig:
        config=self.config.model_evalution
        create_dir([config.root_dir])
        model_evalution_config = ModelEvalutionConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            tokenizer_path = Path(config.tokenizer_path),
            prediction_results=Path(config.prediction_results)
        )
        return model_evalution_config

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import csv

/home/ayush/Documents/AI/Projects/Fake-news-detector/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelEvalution:
    def __init__(self,config=ModelEvalutionConfig):
        self.config=config
        self.model_path = self.config.model_path
        self.tokenizer_path=self.config.tokenizer_path
        # self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_path)
        # self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
          # ⬇️ FIXED: add local_files_only=True
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_path, local_files_only=True)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path, local_files_only=True)
        self.model.eval()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        if not os.path.exists(self.config.prediction_results):
            with open(self.config.prediction_results, mode='w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["text", "label", "confidence"])
    def predict(self,text):
        # Tokenize
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)

        # Predict
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)
            prediction = torch.argmax(probs, dim=-1).item()
            confidence = probs[0][prediction].item()

        # Output
        label = "Real News" if prediction == 1 else "Fake News"
        print(f"📰 Text: {text}")
        print(f"📢 Prediction: {label} ({probs[0][prediction].item():.4f} confidence)")
        with open(self.config.prediction_results, mode='a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([text, label, confidence])

In [8]:
config=ConfigurationManager()
getmodelevalConfig = config.getModelEvalution()
modelEval = ModelEvalution(config=getmodelevalConfig)
modelEval.predict("India successfully lands Chandrayaan-3 near the Moon's south pole")

[2025-06-12 08:16:27,252 : INFO : common : Created directory at: artifact]
[2025-06-12 08:16:27,257 : INFO : common : Created directory at: artifact/model_evalution]


2025-06-12 08:16:32.097285: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 08:16:32.119370: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 08:16:32.289786: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 08:16:32.450123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749696392.603853   32093 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749696392.65

📰 Text: India successfully lands Chandrayaan-3 near the Moon's south pole
📢 Prediction: Real News (1.0000 confidence)
